In [59]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import time
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
import random
from scipy.sparse import hstack
from itertools import combinations
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier


In [2]:
!pip install ipython

In [60]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [61]:
train2=pd.read_csv('train2.csv')
train1=pd.read_csv('train1.csv')
train=pd.concat([train1,train2])
df_all=train
test=pd.read_csv('test_original.csv')

In [62]:
print('CIKM',df_all[df_all['label']=='CIKM']['label'].count())
print('CHI',df_all[df_all['label']=='CHI']['label'].count())
print('KDD',df_all[df_all['label']=='KDD']['label'].count())
print('SIGIR',df_all[df_all['label']=='SIGIR']['label'].count())
print('SIGCSE',df_all[df_all['label']=='SIGCSE']['label'].count())
print('WWW',df_all[df_all['label']=='WWW']['label'].count())
print('siggraph',df_all[df_all['label']=='SIGGRAPH']['label'].count())

CIKM 1673
CHI 1377
KDD 986
SIGIR 1440
SIGCSE 1487
WWW 1215
siggraph 1443


In [63]:
rev_train,labels_train=train['abstract'],train['label']
rev_test,labels_test=test['abstract'],test['label']
counter = CountVectorizer()
counter.fit(rev_train)
counts_train = counter.transform(rev_train)#transform the training data
counts_test = counter.transform(rev_test)#transform the testing data


In [64]:

counts_train = hstack((counts_train,np.array(train['year'])[:,None]))
counts_test = hstack((counts_test,np.array(test['year'])[:,None]))
counts_train = hstack((counts_train,np.array(train['number_author'])[:,None]))
counts_test = hstack((counts_test,np.array(test['number_author'])[:,None]))
counts_train = hstack((counts_train,np.array(train['new_author_count'])[:,None]))
counts_test = hstack((counts_test,np.array(test['new_author_count'])[:,None]))

In [65]:
#train classifier
model1 = MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None)
model2 = LogisticRegression(solver='liblinear')
# model3 = DecisionTreeClassifier(criterion='entropy', min_samples_leaf=5)
# model3 = ExtraTreesClassifier(n_estimators=1300, max_depth=None,min_samples_split=2, random_state=0)
# model3 = ExtraTreesClassifier(n_estimators=500, max_depth=None,min_samples_split=2, random_state=0)

model4 = MLPClassifier(hidden_layer_sizes=(12,),max_iter=20, random_state=14, warm_start=True)
model5 = MLPClassifier(hidden_layer_sizes=(9,),max_iter=17, random_state=14, warm_start=True)
model6 = MLPClassifier(hidden_layer_sizes=(15,), random_state=1, max_iter=13, warm_start=True)
model7 = DecisionTreeClassifier()
model8= RandomForestClassifier(n_estimators=2500, n_jobs=-1,criterion="entropy",max_features='auto',random_state=150,max_depth=1000,min_samples_split=160 )
##model3=LogisticRegression(solver='liblinear')
# model9=svm.SVC(gamma='scale')
model10 = AdaBoostClassifier()


In [ ]:
start_time_all = time.clock()
predictors=[('Multinomial NB',model1),
            ('Logistic Regression',model2),
#             ('ExtraTreesClassifier',model3),
            ('MLP1',model4),
            ('MLP2',model5),
            ('MLP3',model6),
            ('DS',model7),
            ('RF',model8),
#             ('SVM',model9),
            ('BOOST',model10),
           ]

#Fit all 6 models and print accuracy
for predictor in predictors:
    start_time = time.clock()
    model = predictor[1]
    model.fit(counts_train,labels_train)
    test_predictions = model.predict(counts_test)
    print(predictor[0]+' Test Accuracy: {0:.1%}'.format(accuracy_score(test_predictions,labels_test)))
    print('predicting time :',time.clock() - start_time, "seconds")
combos = combinations(predictors, 3)

best_combo_models = []
best_accuracy = 0
print('Checking all combinations of models...')
for combo in combos:
    VT=VotingClassifier(combo)

    #train all classifier on the same datasets
    VT.fit(counts_train,labels_train)

    #use hard voting to predict (majority voting)
    pred=VT.predict(counts_test)
    combo_accuracy = accuracy_score(pred,labels_test)

    #print accuracy
    ensemble_models = []
    for c in combo:
        ensemble_models.append(c[0])
    print('Ensemble Models: ',ensemble_models)
    print('Ensemble Accuracy: {0:0.1%}'.format(combo_accuracy))
    
    #Save best combination by accuracy
    if combo_accuracy > best_accuracy:
        best_accuracy = combo_accuracy
        best_combo_models = ensemble_models
    
print('\nBest Ensemble Models: ',best_combo_models)
print('Best Ensemble Accuracy: {0:0.1%}'.format(best_accuracy))


print('predicting time :',time.clock() - start_time_all, "seconds")

C:\Users\dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.
C:\Users\dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  app.launch_new_instance()
C:\Users\dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
C:\Users\dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  app.launch_new_instance()


Multinomial NB Test Accuracy: 71.5%
predicting time : 0.07220739999957004 seconds
Logistic Regression Test Accuracy: 79.8%
predicting time : 12.960276099999646 seconds
MLP1 Test Accuracy: 69.7%
predicting time : 1.7655575000007957 seconds


C:\Users\dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
C:\Users\dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  app.launch_new_instance()


MLP2 Test Accuracy: 17.0%
predicting time : 1.5619985000002998 seconds


C:\Users\dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
C:\Users\dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  app.launch_new_instance()


...

RF Test Accuracy: 60.1%
predicting time : 27.754771300000357 seconds


C:\Users\dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
C:\Users\dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  app.launch_new_instance()


BOOST Test Accuracy: 53.0%
predicting time : 5.281196999999338 seconds
Checking all combinations of models...


C:\Users\dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


Ensemble Models:  ['Multinomial NB', 'Logistic Regression', 'MLP1']
Ensemble Accuracy: 78.7%
Ensemble Models:  ['Multinomial NB', 'Logistic Regression', 'MLP2']
Ensemble Accuracy: 72.7%
Ensemble Models:  ['Multinomial NB', 'Logistic Regression', 'MLP3']
Ensemble Accuracy: 78.9%
Ensemble Models:  ['Multinomial NB', 'Logistic Regression', 'DS']
Ensemble Accuracy: 78.3%
Ensemble Models:  ['Multinomial NB', 'Logistic Regression', 'RF']
Ensemble Accuracy: 73.6%
Ensemble Models:  ['Multinomial NB', 'Logistic Regression', 'BOOST']
Ensemble Accuracy: 74.5%
Ensemble Models:  ['Multinomial NB', 'MLP1', 'MLP2']
Ensemble Accuracy: 67.7%
Ensemble Models:  ['Multinomial NB', 'MLP1', 'MLP3']
Ensemble Accuracy: 76.6%
Ensemble Models:  ['Multinomial NB', 'MLP1', 'DS']
Ensemble Accuracy: 75.7%
Ensemble Models:  ['Multinomial NB', 'MLP1', 'RF']
Ensemble Accuracy: 71.9%
Ensemble Models:  ['Multinomial NB', 'MLP1', 'BOOST']
Ensemble Accuracy: 71.9%
Ensemble Models:  ['Multinomial NB', 'MLP2', 'MLP3']
Ensem